In [1]:
import os
import pylidc as pl
from pylidc.utils import consensus
from configparser import ConfigParser
import matplotlib.pyplot as plt
from statistics import median_high
import numpy as np
import pandas as pd
from pathlib import Path
import preprocessing
from tqdm import tqdm

In [2]:
DataSet_root = "./LIDC-IDRI"
Processed_data_dir = './data'
IMAGE_DIR = '/IMAGE'
MASK_DIR = '/MASK'
mask_thresh = 8

In [7]:
segmented_lung = None
class prep_ds:
    DataSet_root = "./LIDC-IDRI"
    Processed_data_dir = './data'
    IMAGE_DIR = Processed_data_dir+ "/Image"
    MASK_DIR = Processed_data_dir +"/Mask"
    Clean_IMAGE_DIR =Processed_data_dir + '/CleanIMAGE'
    Clean_MASK_DIR = Processed_data_dir +'/CleanMASK'
    mask_thresh = 8

    dataFrame = pd.DataFrame(index=[],columns=['patient_id','nodule_no','slice_no','original_image','mask_image','malignancy','is_cancer','is_clean'])
    def save_dataFrame(self,meta_list):
      #saves list to csv
        tmp = pd.Series(meta_list,index=['patient_id','nodule_no','slice_no','original_image','mask_image','malignancy','is_cancer','is_clean'])
        self.dataFrame = self.dataFrame.append(tmp,ignore_index=True)
    def calc_malignancy(self,nodule):
        list_malignancy = []
        for annotation in nodule:
            list_malignancy.append(annotation.malignancy)
        malignancy = median_high(list_malignancy)
        if(malignancy>3):
            return malignancy,True
        if(malignancy<3):
            return malignancy,False
        else:
            return malignancy,"Ambiguous"

    
    def prepare_dataset(self):
            if not os.path.exists(self.IMAGE_DIR):
                os.makedirs(self.IMAGE_DIR)
            if not os.path.exists(self.MASK_DIR):
                os.makedirs(self.MASK_DIR)
            if not os.path.exists(self.Clean_IMAGE_DIR):
                os.makedirs(self.Clean_IMAGE_DIR)
            if not os.path.exists(self.Clean_MASK_DIR):
                os.makedirs(self.Clean_MASK_DIR)
            patients = os.listdir(DataSet_root)

            for patient in tqdm(patients):
                pid = patient

    
              
                scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == pid).first()

                nodule_annotations = scan.cluster_annotations()
                vol = scan.to_volume()
      
                patient_img_dir = Processed_data_dir + IMAGE_DIR + '/' + pid
                patient_mask_dir = Processed_data_dir + MASK_DIR + '/' + pid
                Path(patient_img_dir).mkdir(parents=True, exist_ok=True)
                Path(patient_mask_dir).mkdir(parents=True, exist_ok=True)


                if(len(nodule_annotations)>0):

                    #nodules present
                    for i,nodule in enumerate(nodule_annotations):

                        mask,cbbox,masks = pl.utils.consensus(nodule,0.5,512)
                        lung_np = vol[cbbox]
                        malignancy,cancer_label = self.calc_malignancy(nodule)
                        for mask_slice in range(mask.shape[2]):

                            if(np.sum(mask[:,:,mask_slice])<=mask_thresh):
                                continue
                            print(lung_np[:,:,mask_thresh].shape)
                            segmented_lung = preprocessing.preprocess_image(lung_np[:,:,mask_slice])
                            segmented_lung[segmented_lung==-0] =0
                            
                            nodule_name =  "Nodule"+ str(i) +"Slice" +  str(mask_slice)
                            mask_name =  "Mask"+ str(i) +"Slice" +  str(mask_slice)
                            # to save pid nodule index slice index mask name malignancy cancer label is data clean of possible cancer
                            meta_list = [pid[-4:],i,mask_slice,patient_img_dir+ '/' +nodule_name,patient_mask_dir+ '/' +mask_name,malignancy,cancer_label,False]
                            self.save_dataFrame(meta_list)
                            print("Writing image to",patient_img_dir+ '/' +nodule_name)
                            np.save(patient_img_dir+ '/' +nodule_name,segmented_lung)
                            np.save(patient_mask_dir+ '/' +mask_name,mask[:,:,mask_slice])
                else:
                    print("no nodules",pid)
                    cleanpatient_img_dir = self.Processed_data_dir + self.Clean_IMAGE_DIR + '/' + pid
                    cleanpatient_mask_dir = self.Processed_data_dir + self.Clean_MASK_DIR + '/' + pid
                    Path(cleanpatient_img_dir).mkdir(parents=True, exist_ok=True)
                    Path(cleanpatient_mask_dir).mkdir(parents=True, exist_ok=True)
                 
                    for slice in range(vol.shape[2]):
                        if(slice>50):
                            break
                        segmented_lung = preprocessing.preprocess_image(lung_np[:,:,mask_thresh])
                        lung_mask = np.zeros_like(segmented_lung)
                        nodule_name = "Nodule"+ str(i) +"Slice" +  str(slice)
                        mask_name = "Mask"+ str(i) +"Slice" +  str(slice)
                        meta_list = [pid[-4:],i,mask_slice,cleanpatient_img_dir+ '/' +nodule_name+".npy",cleanpatient_img_dir+ '/' +mask_name+".npy",0,False,True]
                        np.save(cleanpatient_img_dir+ '/' +nodule_name,segmented_lung)
                        np.save(cleanpatient_img_dir+ '/' +mask_name,mask[:,:,mask_slice])                    
                    
                    
                print("Saving CSV")
                self.dataFrame.to_csv(self.Processed_data_dir + "/dataset.csv",index = False)
                    
                    



                
               
            
            
            
            

In [ ]:
ds = prep_ds()
ds.prepare_dataset()

  0%|                                                                                                                             | 0/70 [00:00<?, ?it/s]

Loading dicom files ... This may take a moment.
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0001/Nodule0Slice86
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0001/Nodule0Slice87
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0001/Nodule0Slice88
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0001/Nodule0Slice89
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0001/Nodule0Slice90
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0001/Nodule0Slice91
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0001/Nodule0Slice92
(512, 512)


  1%|█▋                                                                                                                   | 1/70 [00:12<14:13, 12.38s/it]

Writing image to ./data/IMAGE/LIDC-IDRI-0001/Nodule0Slice93
Saving CSV
Loading dicom files ... This may take a moment.
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0002/Nodule0Slice171
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0002/Nodule0Slice172
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0002/Nodule0Slice173
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0002/Nodule0Slice174
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0002/Nodule0Slice175
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0002/Nodule0Slice176
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0002/Nodule0Slice177
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0002/Nodule0Slice178
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0002/Nodule0Slice179
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0002/Nodule0Slice180
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0002/Nodule0Slice181
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0002/Nodule0Slice182
(512, 512)
Writin

  3%|███▎                                                                                                                 | 2/70 [00:38<22:58, 20.27s/it]

...

Writing image to ./data/IMAGE/LIDC-IDRI-0020/Nodule1Slice194
Saving CSV
Loading dicom files ... This may take a moment.
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0021/Nodule0Slice49
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0021/Nodule0Slice50
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0021/Nodule1Slice60
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0021/Nodule1Slice61
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0021/Nodule1Slice62
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0021/Nodule1Slice63
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0021/Nodule2Slice125
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0021/Nodule2Slice126
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0021/Nodule2Slice127
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0021/Nodule2Slice128
(512, 512)


 30%|██████████████████████████████████▊                                                                                 | 21/70 [06:10<13:40, 16.75s/it]

Writing image to ./data/IMAGE/LIDC-IDRI-0021/Nodule2Slice129
Saving CSV
Loading dicom files ... This may take a moment.
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0022/Nodule0Slice100
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0022/Nodule0Slice101
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0022/Nodule0Slice102
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0022/Nodule0Slice103
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0022/Nodule0Slice104
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0022/Nodule0Slice105
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0022/Nodule0Slice106
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0022/Nodule0Slice107
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0022/Nodule0Slice108
(512, 512)
Writing image to ./data/IMAGE/LIDC-IDRI-0022/Nodule0Slice109
(512, 512)


 31%|████████████████████████████████████▍                                                                               | 22/70 [06:20<11:54, 14.89s/it]

Writing image to ./data/IMAGE/LIDC-IDRI-0022/Nodule0Slice110
Saving CSV
Loading dicom files ... This may take a moment.


In [ ]:
import matplotlib.pyplot as plt
plt.imshow(segmented_image)

In [ ]:
segmented_image